In [3]:
from bs4 import BeautifulSoup
from http.cookiejar import MozillaCookieJar
import os
import cloudscraper

cookie_file = os.path.expanduser("~/cookies.txt")

cookie = MozillaCookieJar(cookie_file)
cookie.load(ignore_expires=True, ignore_discard=True)
session = cloudscraper.CloudScraper()
req = session.get('https://www.kink.com/shoot/24722', cookies=cookie)
soup = BeautifulSoup(req.text, "html.parser")

In [11]:
soup.find('div', 'column shoot-logo').find('a')['href']

'/channel/electrosluts'

In [43]:
import pandas as pd

df = pd.DataFrame(columns=['number', 'channel'])
list_dir = os.listdir('./library/')
df['number'] = list_dir
df['number'] = pd.to_numeric(df['number'], errors='coerce')

# Drop rows with NaN values
df = df.dropna(subset=['number'])
df['number'] = df['number'].astype(int)
df = df.set_index('number')
df = df.sort_index(ascending=False)
df

,channel
number,
105425,NaN
105398,NaN
105396,NaN
105395,NaN
105393,NaN
...,...
10,NaN
9,NaN
4,NaN


In [33]:
def get_genre(num):
    req = session.get(f'https://www.kink.com/shoot/{num}', cookies=cookie)
    soup = BeautifulSoup(req.text, "html.parser")
    if logo := soup.find('div', 'column shoot-logo'):
        return logo.find('a')['href'].split('/')[-1]
    return None

In [45]:
from IPython.core.display_functions import clear_output

for num, row in df.iterrows():
    # new_row = pd.Series(data={'number': num, 'channel': get_genre(num)})
    df.loc[num]['channel'] = [num, get_genre(num)]
    clear_output(wait=False)
    print(f"print {num} ")
df

print 2 


,channel
number,
105425,"[105425, None]"
105398,"[105398, bondage-liberation]"
105396,"[105396, bondage-liberation]"
105395,"[105395, bondage-liberation]"
105393,"[105393, bondage-liberation]"
...,...
10,"[10, hogtied]"
9,"[9, hogtied]"
4,"[4, hogtied]"


In [54]:
df.to_csv(os.path.join('.', 'table.csv'))

In [55]:
pd.read_csv(os.path.join('.', 'table.csv'))

,number,channel
0,105398,bondage-liberation
1,105396,bondage-liberation
2,105395,bondage-liberation
3,105393,bondage-liberation
4,105392,bondage-liberation
...,...,...
1491,10,hogtied
1492,9,hogtied
1493,4,hogtied
1494,3,hogtied


In [50]:
df['channel'] = df.apply(lambda x: x['channel'][-1], axis=1)


In [53]:
df = df.dropna(subset=['channel'])
df

,channel
number,
105398,bondage-liberation
105396,bondage-liberation
105395,bondage-liberation
105393,bondage-liberation
105392,bondage-liberation
...,...
10,hogtied
9,hogtied
4,hogtied


In [60]:
lib_path = os.path.join('.', 'library')
for dir in os.listdir(lib_path):
    file = os.listdir(os.path.join(lib_path, dir))
    file = os.path.join(lib_path, dir, [f for f in file if f.endswith('.nfo')][0])
    print(file)
    break


./library/3762/3762 - 480p.nfo


In [132]:
import xml.etree.ElementTree as ET


def mod_nfo(num, channel):
    lib_path = os.path.join('.', 'library', str(num))
    file_list = os.listdir(lib_path)
    print(file_list)
    if not [f for f in file_list if f.endswith('.nfo')]:
        raise Exception(file_list, str(num)) 
    print([f for f in file_list if f.endswith('.nfo')][0])
    file_path = os.path.join(lib_path, [f for f in file_list if f.endswith('.nfo')][0])

    tree = ET.parse(file_path)

    # Get the root of the tree
    root = tree.getroot()

    # Create a new 'genre' element
    genre = ET.Element('genre')
    genre.text = channel

    # Append the 'genre' to the root (which is 'movie' in this case)
    root.append(genre)

    # Write the changes back to the NFO file

    tree.write(file_path)

    with open(file_path, 'r') as f:
        content = f.read()

    content = '<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n' + content.replace('</genre>','</genre>\n').replace('<genre>','  <genre>')

    with open(file_path, 'w') as f:
        f.write(content)


In [133]:
from time import sleep

for num, row in df.iterrows():
    mod_nfo(num, row['channel'])
    

['105398 - 1080p.mp4', '105398 - 1080p.nfo', '105398 - 1080p.png', '105398.zip']
105398 - 1080p.nfo
['105396 - 1080p.mp4', '105396 - 1080p.nfo', '105396 - 1080p.png', '105396.zip']
105396 - 1080p.nfo
['105395 - 1080p.mp4', '105395 - 1080p.nfo', '105395 - 1080p.png', '105395.zip']
105395 - 1080p.nfo
['105393 - 1080p.mp4', '105393 - 1080p.nfo', '105393 - 1080p.png', '105393.zip']
105393 - 1080p.nfo
['105392 - 1080p.mp4', '105392 - 1080p.nfo', '105392 - 1080p.png', '105392.zip']
105392 - 1080p.nfo
['105334 - 1080p.mp4', '105334 - 1080p.nfo', '105334 - 1080p.png', '105334.zip']
105334 - 1080p.nfo
['105278 - 1080p.mp4', '105278 - 1080p.nfo', '105278 - 1080p.png', '105278.zip']
105278 - 1080p.nfo
['105277 - 1080p.mp4', '105277 - 1080p.nfo', '105277 - 1080p.png', '105277.zip']
105277 - 1080p.nfo
['105272 - 1080p.mp4', '105272 - 1080p.nfo', '105272 - 1080p.png', '105272.zip']
105272 - 1080p.nfo
['105230 - 1080p.mp4', '105230 - 1080p.nfo', '105230 - 1080p.png', '105230.zip']
105230 - 1080p.nfo


In [130]:
df = df.drop(14623)